# <font color=green>Communauto Station Relocation Problem</font>
***

## Import libraries and dataset
1. Import or install libraries and packages to use.<br>
2. Read the Communaute and Users datasets.<br>
3. A function **frame** is created to print the names.

In [1]:
import pandas as pd
import numpy as np
import gurobipy as gb
from gurobipy import *
from math import radians, cos, sin, asin, sqrt

# the frame() function is used to print a string with a frame.
def frame(title):
    print("\n"+"-"*(len(title)+4)+"\n| "+title+" |\n"+"-"*(len(title)+4))

# Change to your own directory
filePath = "./Datasets/"

# Read the files (vehicle and user information)
communauto = pd.read_csv(filePath + "ListStations.csv")
communauto.Sector = communauto.Sector.str.split(".").str[1].transform(lambda x: x.lstrip().rstrip())
users = pd.read_csv(filePath + "Users.csv")
users = users.iloc[:,:11]

# Print the data
frame("ListStations.csv")
display(communauto.head())
frame("Users.csv")
display(users.head())


--------------------
| ListStations.csv |
--------------------


,Latitude,Longitude,Sector,StationID,StationNo,Zone,Station,Status,IsStreetParking,Capacity
0,45.559980,-73.658280,Ahuntsic,186,38,2,Chambord et Fleury,OK,1,1
1,45.556741,-73.663833,Ahuntsic,198,50,2,Parc Ahuntsic,OK,0,10
2,45.551133,-73.641010,Ahuntsic,339,128,2,College Ahuntsic,OK,0,10
3,45.572389,-73.657966,Ahuntsic,360,138,2,Hamelin et Henri-Bourassa,OK,1,1
4,45.543519,-73.663140,Ahuntsic,380,152,2,Tolhurst et Sauve,OK,1,1



-------------
| Users.csv |
-------------


,UserID,Sector,Longitude,Latitude,rentalHours_day1,rentalHours_day2,rentalHours_day3,rentalHours_day4,rentalHours_day5,rentalHours_day6,rentalHours_day7
0,1.0,Ville-Marie,-73.569770,45.499463,8.0,5.0,10.0,8.0,7.5,7.5,7.5
1,2.0,Ville-Marie,-73.565951,45.501572,6.5,7.0,7.5,7.5,5.0,5.5,6.5
2,3.0,Ville-Marie,-73.568855,45.497963,7.5,5.5,6.5,5.5,7.5,7.0,8.5
3,4.0,Ville-Marie,-73.565685,45.511242,4.5,6.0,7.0,7.0,5.0,8.0,8.5
4,5.0,Ville-Marie,-73.576899,45.509518,6.5,5.5,6.0,7.0,5.5,5.0,6.5


***
The below table demonstrates the numbers of **parking lots** and **vehicles** in each sector.

In [2]:
# Create a table to show the numbers of available parking lots and vehicles in each sector
table1 = communauto[["Sector","IsStreetParking"]].groupby(communauto.Sector).sum("IsStreetParking")
table2 = communauto[["Sector"]].groupby(communauto.Sector).count().rename(columns={"Sector":"Vehicle Count"})
table3 = pd.concat([table1, table2], axis=1, join="inner")
table3["IsStreetParking"] = table3["Vehicle Count"] - table3["IsStreetParking"] 
table3 = table3.rename(columns={"IsStreetParking":"Parking Lot Count"})
display(table3)

,Parking Lot Count,Vehicle Count
Sector,,
Ahuntsic,9,25
Anjou,0,3
Boucherville,1,2
Brossard,4,4
Centre-Sud,1,17
Côte-St-Luc,2,3
Côte-des-Neiges,8,19
Greenfield Park,0,1
Griffintown,0,5


***
<p style="background-color:mistyrose;">
    <br>&emsp;&emsp;Choose the <font color=red><b>Target Sector</b></font> and finalize the available parking lots.<br><br>
</p>

In [3]:
# Choose target sector
#targetSector = "Ville-Marie"
targetSector = "Plateau Mt-Royal"

# Narrow down the dataset based on the target Sector we choose
communauto = communauto[communauto.Sector == targetSector]

# Check the total number of cars in this Sector
vehicleQty = communauto.shape[0]

# Only consider the parking lots in this Sector instead of street parking
communauto = communauto[communauto.IsStreetParking == 0]

# Narrow down the user data
users = users[users.Sector==targetSector].reset_index().drop(["index","Sector"],axis = 1)
frame("Users in "+targetSector)
display(users)

#Print the available parking lot list in the target sector
frame("Available Stations (Parking Lot) in "+targetSector)
display(communauto)


-----------------------------
| Users in Plateau Mt-Royal |
-----------------------------


,UserID,Longitude,Latitude,rentalHours_day1,rentalHours_day2,rentalHours_day3,rentalHours_day4,rentalHours_day5,rentalHours_day6,rentalHours_day7
0,1.0,-73.579563,45.520621,6.0,6.5,7.0,5.5,8.0,6.0,7.0
1,2.0,-73.570866,45.538076,7.5,7.0,6.5,6.0,7.5,6.0,7.0
2,3.0,-73.592470,45.532227,6.5,5.0,6.0,6.5,6.0,7.5,7.5
3,4.0,-73.572507,45.515968,8.5,6.5,7.5,6.0,5.5,7.5,7.0
4,5.0,-73.576676,45.518012,7.5,5.5,5.0,6.0,6.0,6.0,6.5
...,...,...,...,...,...,...,...,...,...,...
143,144.0,-73.582056,45.537632,8.0,6.0,5.0,5.5,6.5,8.5,6.5
144,145.0,-73.576982,45.540602,8.5,6.5,7.0,7.0,7.5,6.5,6.0
145,146.0,-73.583679,45.539719,6.0,8.0,6.0,6.5,5.5,6.5,6.5
146,147.0,-73.576264,45.538212,7.5,8.0,6.5,7.0,8.0,5.0,6.0



--------------------------------------------------------
| Available Stations (Parking Lot) in Plateau Mt-Royal |
--------------------------------------------------------


,Latitude,Longitude,Sector,StationID,StationNo,Zone,Station,Status,IsStreetParking,Capacity
186,45.532228,-73.576052,Plateau Mt-Royal,155,3,1,Garnier,OK,0,16
187,45.522400,-73.566500,Plateau Mt-Royal,156,4,1,Parc La Fontaine,OK,0,16
192,45.530969,-73.588722,Plateau Mt-Royal,282,88,1,Parc Laurier (P sur rue),OK,0,16
194,45.536590,-73.575830,Plateau Mt-Royal,310,113,1,Jardin De Lorimier (P sur rue),OK,0,16
238,45.531117,-73.570275,Plateau Mt-Royal,1189,518,1,Centre Sablon,OK,0,16
240,45.518590,-73.567720,Plateau Mt-Royal,2452,597,1,Metro Sherbrooke,OK,0,16
243,45.535254,-73.585036,Plateau Mt-Royal,2604,632,1,Station-zone Laurier-est,OK,0,16
244,45.538732,-73.581007,Plateau Mt-Royal,2611,637,1,Station-zone De Lorimier nord,OK,0,16


***
## Sets and Parameters
Prepare the parameters and sets that we need for this problem.
<br><br>

* Maximum acceptable distance for users to walk to the vehicles.<br>
$M = 3km$<br><br>

* List of all stations.<br>
$StationList = \{Station_1, Station_2, ..., Station_S\}$ <br><br>

* The capacity of each station.<br>
$StationCapacity = \{C_1, C_2, ..., C_S\}$ <br><br>

* The coordinates of each station.<br>
$StationLocation = \{Coordinate_1, Coordinate_2, ..., Coordinate_S\}$<br><br>

* The list of all users<br>
$UserList = \{User_1, User_2, ..., User_U\}$<br><br>

* The coordinates of each user<br>
$UserLocation = \{Coordinate_1, Coordinate_2, ..., Coordinate_U\}$<br><br>

* The rental hours of each users on each day<br>
$Hours = \{H_{11}, H_{12}, ..., H_{1D}, H_{21}, H_{22}, ..., H_{2D}, ..., H_{U1}, H_{U2}, ..., H_{UD}\}$<br><br>

In [4]:
# Set the maximum acceptable walking distance (in kilometer)
M = 3

# Station list
StationList = communauto.Station.to_list()
frame("StationList")
print(StationList)
   
# Capacity of each station
StationCapacity = communauto[["Station","Capacity"]].set_index("Station")
frame("StationCapacity")
display(StationCapacity)

# Location of each station
StationLocation = communauto[["Station","Longitude","Latitude"]].set_index("Station")
frame("StationLocation")
display(StationLocation)

# User list
UserList = users.UserID.map(lambda x: int(x)).to_list()
frame("UserList")
print(UserList)

# User Location
UserLocation = users[["UserID","Longitude","Latitude"]].set_index("UserID")
frame("UserLocation")
display(UserLocation)

# Rental hours for each user
Hours = users.drop(["Longitude","Latitude"], axis = 1).set_index("UserID")
frame("Hours")
display(Hours)


---------------
| StationList |
---------------
['Garnier', 'Parc La Fontaine', 'Parc Laurier (P sur rue)', 'Jardin De Lorimier (P sur rue)', 'Centre Sablon', 'Metro Sherbrooke', 'Station-zone Laurier-est', 'Station-zone De Lorimier nord']

-------------------
| StationCapacity |
-------------------


,Capacity
Station,
Garnier,16
Parc La Fontaine,16
Parc Laurier (P sur rue),16
Jardin De Lorimier (P sur rue),16
Centre Sablon,16
Metro Sherbrooke,16
Station-zone Laurier-est,16
Station-zone De Lorimier nord,16



-------------------
| StationLocation |
-------------------


,Longitude,Latitude
Station,,
Garnier,-73.576052,45.532228
Parc La Fontaine,-73.566500,45.522400
Parc Laurier (P sur rue),-73.588722,45.530969
Jardin De Lorimier (P sur rue),-73.575830,45.536590
Centre Sablon,-73.570275,45.531117
Metro Sherbrooke,-73.567720,45.518590
Station-zone Laurier-est,-73.585036,45.535254
Station-zone De Lorimier nord,-73.581007,45.538732



------------
| UserList |
------------
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148]

----------------
| UserLocation |
----------------


,Longitude,Latitude
UserID,,
1.0,-73.579563,45.520621
2.0,-73.570866,45.538076
3.0,-73.592470,45.532227
4.0,-73.572507,45.515968
5.0,-73.576676,45.518012
...,...,...
144.0,-73.582056,45.537632
145.0,-73.576982,45.540602
146.0,-73.583679,45.539719



---------
| Hours |
---------


,rentalHours_day1,rentalHours_day2,rentalHours_day3,rentalHours_day4,rentalHours_day5,rentalHours_day6,rentalHours_day7
UserID,,,,,,,
1.0,6.0,6.5,7.0,5.5,8.0,6.0,7.0
2.0,7.5,7.0,6.5,6.0,7.5,6.0,7.0
3.0,6.5,5.0,6.0,6.5,6.0,7.5,7.5
4.0,8.5,6.5,7.5,6.0,5.5,7.5,7.0
5.0,7.5,5.5,5.0,6.0,6.0,6.0,6.5
...,...,...,...,...,...,...,...
144.0,8.0,6.0,5.0,5.5,6.5,8.5,6.5
145.0,8.5,6.5,7.0,7.0,7.5,6.5,6.0
146.0,6.0,8.0,6.0,6.5,5.5,6.5,6.5


***
## Define Functions
two functions **Haversine** and **ManhattanDist** are difined to calculate the distance between two location.

* $ManhattanDist(Coordinates_{user}, Coordinates_{station})$ will return the Manhattan distance (in km) between two locations considering Haversine function. The distance between a user $u$ and a station $s$ is denoted as $D_{us}$

In [5]:
# Calculate the distance between two location using haversine ans manhattan distance
def Haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers.
    return c * r

def ManhattanDist(lon1, lat1, lon2, lat2):
    return(Haversine(lon1, lat1, lon2, lat1) + Haversine(lon2, lat2, lon2, lat1))

#Example
#ManhattanDist(StationLocation.loc["Metro Square-Victoria"]["Longitude"],
#              StationLocation.loc["Metro Square-Victoria"]["Latitude"],
#              StationLocation.loc["Metro Bonaventure"]["Longitude"],
#              StationLocation.loc["Metro Bonaventure"]["Latitude"])

***
## Indice
Set the indice for this problem.
<br><br>

* $s$ : stations,&emsp;$s = 1, 2, ..., S$&emsp;$(s \in StationList)$
* $u$ : users,&emsp;$u = 1, 2, ..., U$&emsp;$(u \in UserList)$
* $v$ : vehicles,&emsp;$v = 1, 2, ..., V$
* $d$ : days,&emsp;$d = 1, 2, ..., D$

In [6]:
V = vehicleQty
S = len(StationList)
U = len(UserList)
D = len(Hours.columns)

print("V for Vehicle: for all v = 1 , ... ,",V)
print("S for Station: for all s =",StationList[0],", ... ,",StationList[S-1],"(total "+str(S)+")")
print("U for User: for all u =",UserList[0],", ... ,",UserList[U-1])
print("D for Day: for all d = 1 , ... ,",D)

V for Vehicle: for all v = 1 , ... , 80
S for Station: for all s = Garnier , ... , Station-zone De Lorimier nord (total 8)
U for User: for all u = 1 , ... , 148
D for Day: for all d = 1 , ... , 7


***
## Model
<br><br>
**1. Created the model for Gurobi.**

In [7]:
# 1 - Model
prob = gb.Model("Communauto Vehicle Allocation")
frame(prob.getAttr("ModelName"))

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-14

---------------------------------
| Communauto Vehicle Allocation |
---------------------------------


***
**2. Add decision variables**

* $X_{ud}$ : if user $u$ successfully books a vehicle on day $d$<br><br>
* $Y_{vs}$ : if vehicle $v$ is allocated to station $s$<br><br>
* $Z_{uv}^d$ : if vehicle $v$ is assigned to user $u$ on day $d$<br>

In [8]:
# 2 - Decision Variable
X = prob.addVars(UserList,D,
                name = ["User "+str(u)+" successfully books a vehicle on day "+str(d+1) 
                        for u in UserList for d in range(D)], vtype = GRB.BINARY)
Y = prob.addVars(V,StationList,
                name = ["Is vehicle "+str(v+1)+" assigned to station "+str(s) 
                        for v in range(V) for s in StationList], vtype = GRB.BINARY)
Z = prob.addVars(UserList,V,D,
                name = ["User "+str(u)+" gets vehicle "+str(v+1)+" on day "+str(d+1) 
                        for u in UserList for v in range(V) for d in range(D)], vtype = GRB.BINARY)

***
**3. Set objective function:**
<br><br>
Maximize profit = total rental hours
<br><br>
$
Maximize\ \sum \limits _{d=1} ^D \sum \limits _{u=1} ^U X_{ud}H_{ud}
$

In [9]:
# 3 - Objective
prob.setObjective(sum(X[u,d]*Hours.loc[u][d] for u in UserList for d in range(D)),GRB.MAXIMIZE)

***
**4. Add constraints**
<br>

* **Constraint (1):** A user can rent at most one vehicle per day.<br><br>
$\sum \limits _{v=1} ^V Z_{uv}^d \le 1,\ \ \forall\ \ u \in UserList,\ \ d \in \{1, 2, ..., D\}$

In [10]:
# 4 - Constrtaints
# Contraint (1)
for u in UserList:
    for d in range(D):
        prob.addConstr(sum(Z[u,v,d] for v in range(V)) <= 1)

***
* **Constraint (2):** A vehicle can be assigned to at most one user per day.<br><br>
$\sum \limits _{u=1} ^U Z_{uv}^d \le 1,\ \ \forall\ \ v \in \{1, 2, ..., V\},\ \ d \in \{1, 2, ..., D\}$

In [11]:
# Contraint (2)
for v in range(V):
    for d in range(D):
        prob.addConstr(sum(Z[u,v,d] for u in UserList) <= 1)

***
* **Constraint (3):** Each vehicle must be allocated to exactly one station.<br><br>
$\sum \limits _{s=1} ^S Y_{vs} = 1,\ \ \forall\ \ v \in \{1, 2, ..., V\}$

In [12]:
# Contraint (3)
for v in range(V):
    prob.addConstr(sum(Y[v,s] for s in StationList) == 1)

***
* **Constraint (4):** Each station should have at least one vehicle.<br><br>
$\sum \limits _{v=1} ^V Y_{vs} \ge 1,\ \ \forall\ \ s \in StationList$

In [13]:
# Contraint (4)
for s in StationList:
    prob.addConstr(sum(Y[v,s] for v in range(V)) >= 1)

***
* **Constraint (5):** The number of vehicles in one station shouldn't exceed the capacity.<br><br>
$\sum \limits _{v=1} ^V Y_{vs} \le C_s,\ \ \forall\ \ s \in StationList$

In [14]:
# Contraint (5)
for s in StationList:
    prob.addConstr(sum(Y[v,s] for v in range(V)) <= StationCapacity.loc[s][0])

***
* **Constraint (6):** If there's a vehicle assigned to a user on one day, then the user successfully books a vehicle on the exact day.<br><br>
$\sum \limits _{v=1} ^V Z_{uv}^d \le X_{ud},\ \ \forall\ \ u \in UserList,\ \ d \in \{1, 2, ..., D\}$

In [15]:
# Contraint (6)
for u in UserList:
    for d in range(D):
        prob.addConstr(sum(Z[u,v,d] for v in range(V)) == X[u,d])

***
* **Constraint (7):** If a user is too far from the station, there won't be any booking.<br><br>
$D_{us}(Z_{uv}^d + Y_{vs} - 1)\le M,\ \ \forall\ \ u \in UserList,\ \ v \in \{1, 2, ..., V\},\ \ d \in \{1, 2, ..., D\},\ \ s \in StationList$

In [16]:
# Contraint (7)
for u in UserList:
    for v in range(V):
        for d in range(D):
            for s in StationList:
                dist = ManhattanDist(UserLocation.loc[u]["Longitude"],
                                    UserLocation.loc[u]["Latitude"],
                                    StationLocation.loc[s]["Longitude"],
                                    StationLocation.loc[s]["Latitude"])
                prob.addConstr(dist * (Z[u,v,d] + Y[v,s] - 1) <= M)

In [17]:
# 5 - Solve
prob.Params.LogToConsole = 0 # To suppress console output
prob.optimize()

In [18]:
# 6 - Result
print("The maximum total rental hours = {:.0f}\n".format(prob.objVal))
for v in prob.getVars():
    if(v.x != 0)&(v.varname.startswith("Is vehicle")):
        print(v.varname,"=",round(v.x))

The maximum total rental hours = 4087

Is vehicle 1 assigned to station Station-zone De Lorimier nord = 1
Is vehicle 2 assigned to station Parc La Fontaine = 1
Is vehicle 3 assigned to station Parc La Fontaine = 1
Is vehicle 4 assigned to station Parc Laurier (P sur rue) = 1
Is vehicle 5 assigned to station Station-zone De Lorimier nord = 1
Is vehicle 6 assigned to station Garnier = 1
Is vehicle 7 assigned to station Parc La Fontaine = 1
Is vehicle 8 assigned to station Station-zone De Lorimier nord = 1
Is vehicle 9 assigned to station Parc La Fontaine = 1
Is vehicle 10 assigned to station Garnier = 1
Is vehicle 11 assigned to station Parc Laurier (P sur rue) = 1
Is vehicle 12 assigned to station Parc Laurier (P sur rue) = 1
Is vehicle 13 assigned to station Parc La Fontaine = 1
Is vehicle 14 assigned to station Station-zone De Lorimier nord = 1
Is vehicle 15 assigned to station Jardin De Lorimier (P sur rue) = 1
Is vehicle 16 assigned to station Metro Sherbrooke = 1
Is vehicle 17 assi

In [19]:
allocation = {}
for s in StationList:
    allocation[s] = 0
for v in range(V):
    for s in StationList:
        allocation[s] += round(Y[v,s].x)
allocation = pd.DataFrame(pd.Series(allocation)).reset_index().rename(columns={"index":"Station",0:"Vehicle Count"})
allocation = pd.merge(allocation,StationLocation,how = "left",left_on = "Station", right_on = "Station")
frame("Optimized Vehicle Allocation")
display(allocation)


--------------------------------
| Optimized Vehicle Allocation |
--------------------------------


,Station,Vehicle Count,Longitude,Latitude
0,Garnier,16,-73.576052,45.532228
1,Parc La Fontaine,16,-73.566500,45.522400
2,Parc Laurier (P sur rue),16,-73.588722,45.530969
3,Jardin De Lorimier (P sur rue),15,-73.575830,45.536590
4,Centre Sablon,4,-73.570275,45.531117
5,Metro Sherbrooke,1,-73.567720,45.518590
6,Station-zone Laurier-est,1,-73.585036,45.535254
7,Station-zone De Lorimier nord,11,-73.581007,45.538732


***
## Visualization
<font color=blue>Result visualization</font>

In [20]:
# You might need to run below line to install the package for the first time.
#!pip install folium
import folium

dfMap = pd.read_csv(filePath + "ListStations.csv")
dfMap.Sector = dfMap.Sector.str.split(".").str[1].transform(lambda x: x.lstrip().rstrip())

loc_center = [dfMap.Latitude.mean(), dfMap.Longitude.mean()]
mapCommunauto = folium.Map(location = loc_center, tiles = "cartodbpositron" ,zoom_start = 11, control_scale = True)

for index, loc in dfMap.iterrows():
    if loc["Sector"] == targetSector:
        color = "crimson"
        folium.Marker([loc["Latitude"], loc["Longitude"]],
                      icon=folium.Icon(icon="car",prefix="fa",color="red"),
                     tooltip="Station: <b>"+loc["Station"]+"<b>",
                     popup=folium.Popup("<small><b>"+loc["Station"]+"</b><br>"+str(loc["Latitude"])+",<br>"+str(loc["Longitude"])+"<small>"),min_width=len(loc["Station"])*40).add_to(mapCommunauto)
    """else:
        color = "lightseagreen"
        folium.CircleMarker([loc["Latitude"], loc["Longitude"]], radius = 2, weight = 5, 
                        fill = True, fill_color = color, color = color).add_to(mapCommunauto)
"""
folium.TileLayer("cartodbpositron").add_to(mapCommunauto)
folium.TileLayer("cartodbdark_matter").add_to(mapCommunauto)
folium.LayerControl().add_to(mapCommunauto)
frame("Current Vehicle Location")
print("Vehicles in target Sector:",targetSector,"are labeled in red.")
mapCommunauto


----------------------------
| Current Vehicle Location |
----------------------------
Vehicles in target Sector: Plateau Mt-Royal are labeled in red.


In [21]:
loc_center = [allocation.Latitude.mean(), allocation.Longitude.mean()]
mapAllocation = folium.Map(location = loc_center, tiles = "cartodbpositron" ,zoom_start = 14, control_scale = True)

# Add allocation result
color = "crimson"
for index, loc in allocation.iterrows():
    folium.CircleMarker([loc["Latitude"], loc["Longitude"]], radius = loc["Vehicle Count"]*8, weight = 0, 
                        fill = True, fill_color = color, color = color,
                        tooltip="Station: <b>"+loc["Station"]+"</b><br>Vehicle Count: <b>"+str(loc["Vehicle Count"])+"</b>",
                        popup=folium.Popup("<small><b>"+loc["Station"]+"</b><br>"+str(loc["Latitude"])+",<br>"+str(loc["Longitude"])+"<small>"),min_width=len(loc["Station"])*40).add_to(mapAllocation)

# Add user location
color = "slategrey"
for index, loc in users.iterrows():
    folium.CircleMarker([loc["Latitude"], loc["Longitude"]], radius = 2, weight = 5, 
                        fill = True, fill_color = color, color = color,
                        tooltip="User ID: <b>"+str(int(loc["UserID"]))+"<b>").add_to(mapAllocation)

folium.TileLayer("cartodbpositron").add_to(mapAllocation)
folium.TileLayer("cartodbdark_matter").add_to(mapAllocation)
folium.LayerControl().add_to(mapAllocation)
frame("Optimized Vehicle Allocation in "+targetSector)
print("Grey dots indicate user locations.")
print("Red circles are the parking lots (the more vehicles it has, the larger it is).")
mapAllocation


----------------------------------------------------
| Optimized Vehicle Allocation in Plateau Mt-Royal |
----------------------------------------------------
Grey dots indicate user locations.
Red circles are the parking lots (the more vehicles it has, the larger it is).
